In [ ]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [ ]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [ ]:
from ipywidgets import Label, HBox, VBox
from riskflow_widgets import Table

In [ ]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))


In [ ]:
import ipywidgets
dir(ipywidgets)

In [ ]:
factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
vol = riskflow_jupyter.rf.riskfactors.construct_factor(factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [ ]:
import k3d
import json


def to_json(string):
    """converts a string to json - skips the whitespace for a smaller output"""
    return json.dumps(string, separators=(',', ':'))


def load_table_from_vol(vol):
    sparse_matrix = {}
    for k in vol.param['Surface'].array:
        sparse_matrix.setdefault(k[0], {}).setdefault(k[1], k[2])

    df = pd.DataFrame(sparse_matrix).sort_index(0, 1)

    table = np.vstack([[0.0]+df.columns.values.round(5).tolist(), 
               np.hstack([df.index.values.round(5).reshape(-1,1).tolist(),
                          df.round(5).replace({np.nan:None})])]).tolist()
    
    return table


class Three(HBox):
    def __init__(self, description):
        self.description = Label(value=description)        
        self.plot = k3d.plot(
            axes=['log(moneyness)', 'expiry','vol(\%)'],
            # menu_visibility=False,
            camera_rotate_speed=3.0
            )
        self.mesh = None                        
        self.data = Table(description='', settings=to_json({
            'width':500, 'height':300, 'contextMenu':True, 'minSpareRows':1, 'minSpareCols':1
        }))
        
        super().__init__(
            children=[
                self.description,
                VBox(children=[self.plot, self.data])
            ]
        )

    def observe(self, handler, prop, type='change'):
        
        def make_plot_fn(fn):
            def update_plot(change):
                fn(change)
                self.update_plot(json.loads(change['new']))
            return update_plot
           
        if self.children:
            self.data.observe(make_plot_fn(handler), prop, type)
        else:
            return super().observe(handler, prop, type)
        
    @staticmethod
    def make_faces_vectorized1(Nr,Nc):

        out = np.empty((Nr-1,Nc-1,2,3),dtype=int)

        r = np.arange(Nr*Nc).reshape(Nr,Nc)

        out[:,:, 0,0] = r[:-1,:-1]
        out[:,:, 1,0] = r[:-1,1:]
        out[:,:, 0,1] = r[:-1,1:]

        out[:,:, 1,1] = r[1:,1:]
        out[:,:, :,2] = r[1:,:-1,None]

        out.shape =(-1,3)
        return out
    
    @staticmethod
    def interpolate_surface(json_list):
        moneyness = json_list[0][1:]
        e = []
        for p in json_list[1:]:
            e.extend([[m, p[0], v] for m,v in zip(moneyness, p[1:]) if v is not None])

        surface = np.array(e)
        expiry = [p[0] for p in json_list[1:]]
        return expiry, moneyness, np.array([np.interp(
            moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])

    def update_plot(self, json_list):
        e, moneyness, vol = Three.interpolate_surface(json_list)
        scale = 2/np.log(2)
        m = scale*np.log(moneyness)
        v = vol*100
        
        U, V = np.meshgrid(m, e)
        vertices = np.dstack([U, V, v]).reshape(-1,3)
        indices = Three.make_faces_vectorized1(*v.shape)
        if self.mesh is not None:
            self.plot-=self.mesh
        self.mesh = k3d.mesh(
            vertices.astype(np.float32), indices.astype(np.uint32), flat_shading=False, attribute=v,
            side='double', color_map=k3d.basic_color_maps.Reds, color_range=[v.min(), v.max()]
        )
        self.plot +=self.mesh
        
    @property
    def value(self):        
        return self.data.value
    
    # a setter function
    @value.setter
    def value(self, json_list):
        self.data.value = to_json(json_list)
        self.update_plot(json_list)


In [ ]:
j = load_table_from_vol(vol)
#np.isnan(None)

In [ ]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test' )



In [ ]:
def handle(change):
    print(change['new'])
    
    
t.observe(handle, 'value')


In [ ]:
#pd.DataFrame(json.loads(t.data.value))
t.value = j

In [ ]:
t.data.value

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
fc.selected_filename

In [ ]:
cx

In [ ]:
pp = riskflow_jupyter.PortfolioPage(cx.current_cfg)

In [ ]:
pp.main_container

In [ ]:
rfp = riskflow_jupyter.RiskFactorsPage(cx.current_cfg)

In [ ]:
rfp.main_container


In [ ]:
rfp.tree.observeserve??


In [ ]:
%debug

In [ ]:
t=rfp.data['FXVol.EUR.ZAR']['Factor']['Surface']['value']
#rfp.data['FXVol.EUR.ZAR']['Factor']['Surface']
#rfp.right_container.children[0].children[4].value=t
rfp.right_container.children[0].children[4].data.observe(lambda x:print(x['new']), 'value')

In [ ]:
rfp.right_container.children[0].children[4].data